# Build the multi-modal for intermodal route planning

**Nodes**: Stations in the SBB traffic network. Curated from the following
sources:

- `regional_station.csv` from
  [here](https://data.sbb.ch/explore/dataset/regionale-fahrplane/information/)

- `city_station.csv` from
  [here](https://data.sbb.ch/explore/dataset/stadtefahrplan/information/)

- `pr_station.csv` from
  [here](https://data.sbb.ch/explore/dataset/mobilitat/information/)

Each node has the `station_name` as the primary attribute and then also
information about the location of the station (`pos` (tuple of lon, lat float),
`loc` (string repr of the former), `opucid` (an identifier) and `abbr` (the
abbreviation of the station name)).

**Edges**: Trip segments between stations for multiple modalities (foot, bike,
car, train). The latter sourced from the timetable of the SBB for a given day
(see `timetable.ipynb` for details). The former are generated from the `nodes`
by computing all pairs of nodes that are within a given time travel threshold
(e.g. all stations within 30 minutes foot distance).

Three types of nodes:

- Static for modalities `foot`, `bike` and `car` from each node given a time
  threshold
- Dynamic given start position for all modalities given with k-nearest neighbors
- ...

Results: Directed, multi-edged graph


In [1]:
%reload_ext autoreload
%autoreload 2

# Immports
import numpy as np
import pandas as pd
import networkx as nx

from tqdm import tqdm
import pickle
import datetime

from matplotlib import pyplot as plt
import cartopy as ccrs
import cartopy.feature as cfeature

import sys
sys.path.append('../')

import utils as utils

In [2]:
# Load the data
df = pd.read_csv("../data/ist-daten-sbb.csv", sep=";")

# Select and rename columns
cols = {
    "Journey identifier": "journey_id",
    "Arrival time": "arrival",
    "Departure time": "departure",
    "Stop name": "station",
    "OPUIC": "opuic",
    "Geopos": "pos",
}

# Select and rename columns
df = df[cols.keys()].rename(columns=cols)

# Convert time columns to datetime
df["arrival"] = pd.to_datetime(df["arrival"])
df["departure"] = pd.to_datetime(df["departure"])

# Show the result
df.head()

,journey_id,arrival,departure,station,opuic,pos
0,85:11:1094:001,2023-12-02 00:06:00,2023-12-02 00:07:00,Thun,8507100,"46.75485273059273, 7.6296058286694795"
1,85:11:1096:001,2023-12-02 00:37:00,NaT,Basel SBB,8500010,"47.5474120550501, 7.589562790156525"
2,85:11:1251:001,NaT,2023-12-01 06:06:00,Basel SBB,8500010,"47.5474120550501, 7.589562790156525"
3,85:11:1258:001,NaT,2023-12-01 20:08:00,Chur,8509000,"46.853084162764006, 9.52893773304132"
4,85:11:1411:001,NaT,2023-12-01 07:10:00,Bern,8507000,"46.948832290498416, 7.439130889923935"


## Add Route Network from SBB Timetable

Using data from `01.12.2023`


In [3]:
# Build edge list (with edge attributes) for train journeys
edges = []
for journey_id in tqdm(df.journey_id.unique()):
    trip = df[df.journey_id == journey_id].sort_values("departure", inplace=False)
    trip_name = f"{trip.iloc[0].station} -> {trip.iloc[-1].station}"

    for i in range(len(trip) - 1):
        edges.append(
            (
                trip.iloc[i].station,
                trip.iloc[i + 1].station,
                {
                    "departure": trip.iloc[i].departure,
                    "arrival": trip.iloc[i + 1].arrival,
                    "duration": trip.iloc[i + 1].arrival - trip.iloc[i].departure,
                    "journey_id": journey_id,
                    "trip_name": trip_name,
                    "type": "train",
                },
            )
        )

100%|██████████| 5726/5726 [00:26<00:00, 220.17it/s]


In [4]:
# Initialise multi-graph object
G = nx.MultiDiGraph(edges)

# We have the following attributes for each edge (for train journeys)
list(G.edges(data=True))[0]

('Interlaken Ost',
 'Interlaken West',
 {'departure': Timestamp('2023-12-01 23:33:00'),
  'arrival': Timestamp('2023-12-01 23:36:00'),
  'duration': Timedelta('0 days 00:03:00'),
  'journey_id': '85:11:1094:001',
  'trip_name': 'Interlaken Ost -> Bern',
  'type': 'train'})

In [5]:
# Add node attributes (station id and position)
unique_stations = df.drop_duplicates(subset=["station"])
node_attrs = {
    row.station: {"opuic": row.opuic, "pos": row.pos}
    for _, row in unique_stations.iterrows()
}

nx.set_node_attributes(G, node_attrs)

Add information about whether station has parking spot (`has_pr`)


In [6]:
# Read in city and regional stations
pr_stations = pd.read_csv("../data/pr_stations.csv")

# Rename columns
pr_stations = pr_stations.rename(columns={"station": "name", "station_abbr": "abbr"})

# Initialise has_pr to False for all nodes
for node in G.nodes:
    G.nodes[node]["has_pr"] = False

not_found = 0
for pr_station in pr_stations.name.unique():
    if pr_station in G.nodes:
        G.nodes[pr_station]["has_pr"] = True
    else:
        not_found += 1

print(
    f"Added {pr_stations.name.nunique() - not_found} of {pr_stations.name.nunique()} PR stations. {not_found} not found."
)

Added 409 of 523 PR stations. 114 not found.


In [7]:
# We have the following attributes for each node
list(G.nodes(data=True))[0]

('Interlaken Ost',
 {'opuic': 8507492,
  'pos': '46.690499996187924, 7.869000004346448',
  'has_pr': False})

In [8]:
# Check that we have position for all stations
[node for node, attr in G.nodes(data=True) if attr["pos"] is np.nan]

['Pino-Tronzano', 'Maccagno', 'Colmegna', 'Lottstetten', 'Jestetten']

## Grow network to include other modalities

We now include the following modalities: `foot`, `bike`, `car`, `train`.


In [12]:
# Utility to grow a graph from a list of nodes

# Set limits km/h
avg_speed = {
    "foot": 1,  # m/s
    "bike": 2,  # m/s
    "car": 10,  # m/s
}

limits = {
    "foot": 900,  # s (30min)
    "bike": 900,  # s (1h)
    "car": 900,  # s (1h)
}

# Iterate over all pairs of nodes and mode types (V*V*num_modes)
added_edges = {
    "foot": 0,
    "bike": 0,
    "car": 0,
}
for u, attr_u in tqdm(G.nodes(data=True)):
    for v, attr_v in G.nodes(data=True):
        # Discard if same node
        if u == v:
            continue

        # Get longitude and latitude for start and end station
        u_pos = attr_u["pos"]
        v_pos = attr_v["pos"]

        # Discard if no position available
        if u_pos is np.nan or v_pos is np.nan:
            continue

        for mode in ["foot", "bike", "car"]:
            # Discard if air distance is above thresholds (not reachable with mode)
            dist = utils.get_distance(u_pos, v_pos)  # dist in m
            if dist > limits[mode] * avg_speed[mode]:
                continue

            # Otherwise compute travel time for mode and add edge if below threshold
            time = utils.get_exact_travel_time(u_pos, v_pos, mode)
            if time < limits[mode]:
                # print(f"Adding from {u} to {v} via {mode} in {(time/60):.2f}min (less than {limits[mode]/60:.2f}min)")
                G.add_edge(u, v, mode=mode,
                           duration=datetime.timedelta(seconds=time))
                added_edges[mode] += 1

print(f"Added {added_edges['foot']} foot edges.")
print(f"Added {added_edges['bike']} bike edges.")
print(f"Added {added_edges['car']} car edges.")

  0%|          | 0/603 [00:00<?, ?it/s]

  0%|          | 0/603 [00:00<?, ?it/s]


KeyError: 'route'

## Visualise graph


In [ ]:
# Set up the map projection and the transformation
proj = ccrs.Mercator()
transform = ccrs.Geodetic()

# Create a figure with an axes set with the projection
fig, ax = plt.subplots(subplot_kw={"projection": proj}, figsize=(30, 10))

# Set the extent of the map (min longitude, max longitude, min latitude, max latitude)
ax.set_extent([5, 12, 45.5, 48], crs=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=":")

# Draw nodes (use scatter for individual node plotting)
for _, attr in G.nodes(data=True):
    lon, lat = attr["pos"]
    col = "red" if attr["has_pr"] else "blue"
    ax.scatter(lat, lon, s=10, color=col, transform=transform, zorder=3)

# Draw edges (use scatter for individual edge plotting)
# for edge in G.edges():
#    lons, lats = zip(*[pos[node] for node in edge])
#    ax.plot(lons, lats, color='gray', linewidth=2, transform=transform, zorder=2)

# Add node labels
# for node, (lon, lat) in pos.items():
#   plt.text(lon-0.02, lat-0.015, node, transform=transform, horizontalalignment='right')


print("All city and regional stations")
plt.show()

### Save the graph


In [ ]:
with open("../data/graph.pickle", "wb") as file:
    pickle.dump(G, file)